# Predicting Heart Disease with Age, Serum Cholesterol, Resting Blood Pressure and Maximum Heart Rate


In [1]:
download from keggle 
check seeds 
check spellings

ERROR: Error in parse(text = x, srcfile = src): <text>:1:10: unexpected symbol
1: download from
             ^


## Introduction
Heart disease is a disease that refers to several types of heart conditions, which is the main cause of death in the USA for most of racial and ethnic groups (1). The main risk factors for heart disease are high blood pressure, high blood cholesterol, smoking, and age. Some of the common heart disease symptoms are chest pain, irregular heartbeats (heart beats too fast or too slow or irregularly), Pain in the back, and shortness of breath. A doctor usually diagnose a patients as either healthy or heart diseased based on their risk factors and symptoms; as a result, a doctor may make prediction about heart disease based on the current situation and his or her past experiment. The Heart disease data was downloaded from Kaggle to use for prediction of heart disease. To answer this question, we will use the “Heart Failure Prediction Dataset”. This dataset contains 11 variables, which Age, RestingBP, Cholesterol, MaxHR variables will be used to predict the heart disease status (1 = heart disease and 0 = normal).



#### Question: 
   Can we use the Age, normal blood pressure when body is at rest, maximum heart rate achieved, and blood cholesterol level variables to predict if a patient heart disease or not? 
   



## Methods & Results 

In [1]:
### Loading Libraries into R
library(tidyverse)
library(tidymodels)
library(cowplot)

library(repr)
options(repr.matrix.max.rows = 6)

#set the seed
set.seed(4321) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

### 1. Loading the Data
The Heart disease data was downloaded from Kaggle as a CSV file and was saved to the 'data' folder.

???? fix it, should not be from kaggle

In [2]:
#Reading data from the "data" folder, no data explaination on the csv file, no need for skipping. 
#data is tidy 
#we read the data into R as "data"
set.seed(123)
heart_data <- read_csv("data/heart.csv")
heart_data

Rows: 918 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope
dbl (7): Age, RestingBP, Cholesterol, FastingBS, MaxHR, Oldpeak, HeartDisease

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
40,M,ATA,140,289,0,Normal,172,N,0,Up,0
49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
37,M,ATA,130,283,0,ST,98,N,0,Up,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1
38,M,NAP,138,175,0,Normal,173,N,0.0,Up,0


## Variable Explanation 

Age: age of the patient [years]

RestingBP: resting blood pressure [mm Hg]

Cholesterol: serum cholesterol [mm/dl]

MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]

HeartDisease:[1: heart disease, 0: Normal]

### Wrangling & Cleaning the Data
The data is tibble and the `Cholesterol` = 0 is equivalent to NA in this data set. Except`Cholesterol`, all the other variables do not contain NA. We are selecting for these following numerical variables as our predictors for the model: 
age, resting blood pressure (mmHg), maximum heart rate, and serum cholesterol level (mm/dl).

In [3]:
heart_data|>
        filter(Cholesterol==0)|>
        group_by(Cholesterol)|>
        summarize(n=n())

Cholesterol,n
<dbl>,<int>
0,172


We do not have data for blood cholestrol of 172 patients, so the data set is filtered to remove `Cholesterol` = 0

In [4]:
#We filter against NA in Cholesterol and check
heartdisease_data <- heart_data|> 
    filter(Cholesterol!=0)
heartdisease_data

Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
40,M,ATA,140,289,0,Normal,172,N,0,Up,0
49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
37,M,ATA,130,283,0,ST,98,N,0,Up,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1
38,M,NAP,138,175,0,Normal,173,N,0.0,Up,0


In this data set `HeartDisease` was incorrectly stored as a numerical (dbl) column. We use mutate() to turn it into a factor. Then the following variables are selected as these are predictors in this model. 

In [5]:
#Convert the character class variable to the factor data type
data_heart <- heartdisease_data|>
        mutate(HeartDisease = as.factor(HeartDisease))|>
        select(Age, RestingBP, Cholesterol, MaxHR, HeartDisease)
data_heart

Age,RestingBP,Cholesterol,MaxHR,HeartDisease
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
40,140,289,172,0
49,160,180,156,1
37,130,283,98,0
⋮,⋮,⋮,⋮,⋮
57,130,131,115,1
57,130,236,174,1
38,138,175,173,0


## Spliting data
Spliting the data into training set and testing set, 75% of the data in the training set

In [6]:
do we need seed here or the one above is enough
farase said dont put seed in this cell but check again 
check with TA? BUT I do not think so

ERROR: Error in parse(text = x, srcfile = src): <text>:1:4: unexpected symbol
1: do we
       ^


In [7]:
set.seed(123)
heart_split <- initial_split(data_heart, prop = 0.75, strata = HeartDisease) 
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)
                     
heartdisease_proportions <- heart_train |>
    group_by(HeartDisease) |>
    summarize (n = n()) |>
    mutate (percent = 100*n/nrow(heart_train))
                     
heart_train
heartdisease_proportions


Age,RestingBP,Cholesterol,MaxHR,HeartDisease
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
37,130,283,98,0
39,120,339,170,0
45,130,237,170,0
⋮,⋮,⋮,⋮,⋮
45,110,264,132,1
68,144,193,141,1
57,130,131,115,1


HeartDisease,n,percent
<fct>,<int>,<dbl>
0,292,52.23614
1,267,47.76386


In [8]:
explain the heart diseas percentage and n in words

ERROR: Error in parse(text = x, srcfile = src): <text>:1:9: unexpected symbol
1: explain the
            ^


## Table

Table summary of the predictors and their numerical distribution (mean and variance) in each class of the target variable (0 = Normal, 1 = Diseased)

In [ ]:
nheart <- heart_train |>
        group_by(HeartDisease)|>
        summarize(Observation=n())

table_1 <- heart_train|>
    group_by(HeartDisease)|>
    summarize(across(Age:MaxHR, mean, na.rm=TRUE)) 
mean_heart_disease <- merge(table_1,nheart) |>
    rename(Avg_Age = Age, Avg_RestingBP = RestingBP, Avg_Cholesterol = Cholesterol, Avg_MaxHR = MaxHR)

table_2 <- heart_train|>
        group_by(HeartDisease)|>
        summarize(across(Age:MaxHR, var, na.rm=TRUE)) 
variance_heart_disease <- merge(table_2,nheart) |>
    rename(var_Age = Age, var_RestingBP = RestingBP, var_Cholesterol = Cholesterol, var_MaxHR = MaxHR) 


mean_heart_disease
variance_heart_disease

The maen of each variables that are used for the heart disease prediction are measured to demonstrate the data's center point of the distribution of random variables. The variance of each variables are also measured to display the relative relationship of individual observation to each others within the data set. 

In [ ]:
 Double check ????

## Visualization 

Visualizing the data in order to find any potential correlation between predictors and their effect on the classification of the target value.

The variables are also standardized because the axis 

In [ ]:
complete the sentence abve
why do we standarize talk about two things: the mean and the sd


In [ ]:
data_train_recipe <- recipe(HeartDisease ~ ., data= heart_train)|>
    step_scale(all_predictors())|>
    step_center(all_predictors()) |>
    prep()
data_train_recipe

scaled_heart <- bake(data_train_recipe, heart_train)
scaled_heart

In [10]:
options(repr.plot.width = 15, repr.plot.height = 8) 

#scaterplot resting blood pressure and age 
agevsbp <- ggplot(scaled_heart, aes(x = Age, y = Cholesterol, color = HeartDisease)) +
      geom_point( alpha = 0.8) +
      xlab("Age (standarized)") +
      ylab("Serum Cholesterol (standarized) ") +
        labs(color= "Heart Disease Status")+
      theme(text = element_text(size = 15),
        legend.position = "right",
        legend.direction = "vertical") +
        scale_color_manual(labels = c("Healthy","Diseased"),
                           values= c("steelblue2","orange2"))+
        ggtitle("Distribution of the Heart Disease Cases \n Based on Age and Blood Cholesterol") 
 agevsbp     

ERROR: Error in ggplot(scaled_heart, aes(x = Age, y = Cholesterol, color = HeartDisease)): object 'scaled_heart' not found


Figure 1: Scatter plot of Serum Cholesterol versus Age colored by heart disease status label

In [9]:
explain the relationship and whta we expect

ERROR: Error in parse(text = x, srcfile = src): <text>:1:9: unexpected symbol
1: explain the
            ^


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8) 
cholvshr <- ggplot(scaled_heart, aes(x = RestingBP, y = MaxHR, color = HeartDisease)) +
      geom_point(alpha = 0.7) +
      xlab("Resting Blood Pressure (standarized)") +
      ylab("Maximum Heart Rate Achieved \n (standarized)") +
      labs(color= "Heart Disease Status")+
      theme(text = element_text(size = 15),
        legend.position = "right",
        legend.direction = "vertical") +
        scale_color_manual(labels = c("Healthy","Diseased"),
                           values= c("steelblue2","orange2"))+
        ggtitle("Distribution of the Heart Disease Cases \n Based on maximum heart rate and resting blood pressure")
cholvshr

Figure 2: Scatter plot of maximum heart rate achieved versus resting blood pressure  colored by heart disease status label

In [ ]:
explain the relationship and whta we expect

In [ ]:
do we need this percentage below
I dont think so

### Percentage 

Finding the ratio of each class of the target variable (0 = Normal, 1 = Diseased) in the total population.

In [ ]:
n_row <- nrow(heart_data)

data_percentage <- heart_data |>
    group_by(HeartDisease)|>
    summarize(count=n(), percentage=n()/n_row*100)
data_percentage

n_row_train <- nrow(heart_train)

data_train_percentage <- heart_train |>
    group_by(HeartDisease)|>
    summarize(count=n(), percentage=n()/n_row_train*100)
data_train_percentage

### Determining the best k value (number of neighbors)

In [ ]:
 explain this

In [ ]:
set.seed(123)

data_vfold <- vfold_cv(heart_train, v = 10, strata = HeartDisease)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
        set_engine("kknn") |>
        set_mode("classification")

k_vals <- tibble(neighbors= seq(from= 1, to= 100, by= 5))

knn_result <- workflow()|>
    add_recipe(data_train_recipe)|>
    add_model(knn_spec)|>
    tune_grid(resamples=data_vfold, grid=k_vals)|>
    collect_metrics()

accuracies <- knn_result |>
  filter(.metric == "accuracy")

print(accuracies)

In [ ]:
accuracy_vs_k <- ggplot(accuracies, aes(x=neighbors, y=mean))+
    geom_point()+
    geom_line()+
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    theme(text = element_text(size = 15))+
    scale_x_continuous(breaks = seq(0, 100, by = 5))+  
    scale_y_continuous(limits = c(0.6, 0.8), breaks=seq(0.6, 1, by=0.04))+
    ggtitle ("The correclation between number of neighbors and accuracy")
accuracy_vs_k

Figure 3: Plot of estimated accuracy versus the number of neighbors

Setting the number of neighbors to K = 66 gives the highest accuracy (70.66%). However, values after k = 20 have a similar accuracy. Therefore, any selection from k= 20 to k= 95 can be reasonable but k = 66 is provides the highest accuracy. 


### Finding the accuracy of the model

???? explain

In [ ]:
maxmax<- accuracies|>
    filter(mean == max(mean))
maxmax

For prediction of heart disease the accuracy of the 70.66% is not a very good value for this data. Heart disease is very seriouse disease and cause a lot of death yearly so the accuracy for prediction of this data should be high. Therefore, this result suggest that prediction with these variable is not sufficient; however, it is also important to check the confusion matrix. 

In [ ]:
this one above is for k not the model

In [ ]:
set.seed(123)

mnist_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 66) |>
       set_engine("kknn") |>
       set_mode("classification")

mnist_fit <- workflow() |>
            add_recipe(data_train_recipe) |>
            add_model(mnist_spec) |>
            fit(data = heart_train)

#mnist_accurate<- mnist_fit|>
 #   collect_metrics()|>
  #          filter(.metric=="accuracy")



#mnist_accurate

In [ ]:

mnist_predictions <- predict(mnist_fit, heart_test)|>
        bind_cols(heart_test)

mnist_predictions

mnist_metrics <- mnist_predictions |>
        metrics(truth=HeartDisease, estimate = .pred_class)

mnist_metrics

mnist_conf_mat <- mnist_predictions |>
        conf_mat(truth = HeartDisease, estimate = .pred_class)

mnist_conf_mat

In [ ]:
explainn how good our model is

In [ ]:
put graphs toghether

The confusion matrix displays 68 observations were correctly predicted as healthy and 50 were correctly predicted as diseased. However, it shows 39 patients as healthy while they were diseased and 30 patients was predicted as diseased while they wear healthy. Therefore, the classifier predicts 118 (68 + 50) observations correctly, and it determines 69 (30 + 39) observations mistakenly. As a result, this confusion matrix suggests this prediction is not very helpful as 39 patients with heart disease was mistakenly diagnosed as healthy, and they may not receive further tests and treatments which is dangerous.

question could be how accurate could we predit only with ata chest pain

## Discussion:

By developing this model, we expect to be able to predict heart disease, with a reasonable accuracy for application in healthcare, based on the biological determinants and symptoms observed in individuals. In particular, this model predicts the heart disease by taking some of the most important risk factors into consideration. It anticipates the presence, or the absence of the heart disease based on the combination of these numerical variables in a systemic approach, instead of relying on the personal judgement of a healthcare provider.
 Therefore, our model can lay the groundwork for designing a potential diagnostic tool assisting healthcare professionals to address life threatening heart diseases earlier and more accurately. 
Also, in our analysis, some relationships may be observed between heart disease and the predictors.
These observations can lead the researchers to answer further questions like what the physiological
mechanisms behind these relationships are. This model can be expanded by designing a model capable of predicting heart disease in individuals considering different types of chest pains and across different genders, which are the limitations of the current model.
One the possible reasons for the relatively low accuracy of this model can be the potential confounding effects between some of the predictors that we were not able to address due the limitations of the model or the dataset. For example, in Figure 1, we can see that there is no correlation between the cholesterol level and the heart disease, which is not consistent with what is known medically. Thus, maybe the negative effect of high cholesterol in heart diseases can be observed when it is considered along with the amount of exercise (not numerical) or stress levels (not available in the dataset).

## Refrences
1.Tsao CW, Aday AW, Almarzooq ZI, Beaton AZ, Bittencourt MS, Boehme AK, et al. Heart Disease and Stroke Statistics—2022 Update: A Report From the American Heart Association. Circulation. 2022;145(8):e153–e639.

2.Tsao CW, Aday AW, Almarzooq ZI, Beaton AZ, Bittencourt MS, Boehme AK, et al. Heart Disease and Stroke Statistics—2022 Update: A Report From the American Heart Association. Circulation. 2022;145(8):e153–e639.

3.Centers for Disease Control and Prevention, National Center for Health Statistics. About Multiple Cause of Death, 1999–2020. CDC WONDER Online Database website. Atlanta, GA: Centers for Disease Control and Prevention; 2022. Accessed February 21, 2022.

4.fedesoriano. (September 2021). Heart Failure Prediction Dataset. Retrieved [Date Retrieved] from https://www.kaggle.com/fedesoriano/heart-failure-prediction.
